In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
dataset=pd.read_csv("../input/depression-social-media/df_train.csv")
dataset.head()

In [4]:
dataset.shape

In [5]:
dataset["Sentiment"].unique()

In [6]:
list(dataset[dataset["Sentiment"]==3]["Text"])[8]

In [7]:
dataset["Sentiment"]=(dataset["Sentiment"]+1).astype(int)
dataset.head(2)

In [8]:
# dataset=dataset.sample(999)

In [9]:
!pip install torchtext==0.11.2

In [10]:
#deal with tensors
import torch   

#handling text data
# from torchtext import data
import torchtext.legacy.data as data

In [11]:
# !pip install torchtext==0.11.2

In [12]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [13]:
"this is great"
["this","is","great"]

In [14]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [15]:
fields = [(None,None), ('Text',TEXT),('Sentiment', LABEL)]


In [16]:
#loading custom dataset
training_data=data.TabularDataset(path = '../input/depression-social-media/df_train.csv',format = 'csv',fields = fields,skip_header = True)

#print preprocessed text
print(vars(training_data.examples[0]))

In [17]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [18]:
# ["This","is","great"] this - [0.2,0.3,0.5] 

In [19]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3)  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
#print(TEXT.vocab.stoi)   

In [20]:
torch.save(TEXT.vocab, 'vocab_obj.pth')

In [21]:
vocab_obj = torch.load('vocab_obj.pth')

In [22]:
# path="./voc.txt"
# def save_vocab(vocab, path=path):
#     with open(path, 'w+', encoding='utf-8') as f:     
#         for token, index in vocab.stoi.items():
#             f.write(f'{index}\t{token}\n')

In [23]:
# def read_vocab(path="./voc.txt"):
#     vocab = dict()
#     with open(path, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 print(line)
#                 index, token = line.split('\t')
#                 vocab[token] = int(index)
#             except:
#                 continue
#     return vocab

In [24]:
# TEXT.vocab=read_vocab()

In [25]:
# save_vocab(TEXT.vocab)

In [26]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.Text),
    sort_within_batch=True,
    device = device)

In [27]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
#         packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=dense_outputs
        
        return outputs

In [28]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = len(LABEL.vocab)
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [29]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
# pretrained_embeddings = TEXT.vocab.vectors
# model.embedding.weight.data.copy_(pretrained_embeddings)

# print(pretrained_embeddings.shape)

In [30]:
# [0.8,1,5,8,9] [0.1,0.2,0.9,0.001,0.002] 

In [31]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

#define metric
def accuracy(preds, y):
    #round predictions to the closest integer
    
    rounded_preds = torch.argmax(torch.softmax(preds,dim=1),1)
    
    
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [32]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.Text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, torch.from_numpy(np.array(batch.Sentiment)).long())        
        
        #compute the binary accuracy
        acc = accuracy(predictions, batch.Sentiment)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.Text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, torch.from_numpy(np.array(batch.Sentiment)).long())
            acc = accuracy(predictions, batch.Sentiment)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
device

In [35]:
for batch in valid_iterator:
    text, text_lengths = batch.Text   
        
    #convert to 1D tensor
    predictions = model(text, text_lengths).squeeze()  

    #compute the loss
    loss = criterion(predictions, torch.from_numpy(np.array(batch.Sentiment)).long())        

    #compute the binary accuracy
    acc = accuracy(predictions, batch.Sentiment)   
    break

In [40]:
N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print(f"Epoch: {epoch+1}")
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [41]:
#load weights
path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();

#inference 
import spacy
nlp = spacy.load('en_core_web_sm')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
    indexed = [vocab_obj.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction 
    return torch.argmax(torch.softmax(prediction,dim=-1),dim=-1).item()

In [43]:
sample=list(dataset["Text"])[6]
sample

In [39]:
#make predictions
predict(model, "Are there any sports that you don't like?")

#insincere question
predict(model, sample)

In [40]:
list(dataset["Text"])[6]

In [65]:
torch.argmax(torch.softmax(predict(model, "anxiety ruining my life im  years old and still have the same crippling anxiety"),dim=-1),dim=-1)